In [24]:
import os
import re
import json
import subprocess

from openai import OpenAI
import sys
sys.path.append('../')
from prompts.design_prompt import design_prompt
from prompts.parsing_prompt import parsing_prompt
from prompts.scripting_prompt import scripting_prompt

In [25]:
with open('../config/config.json') as f:
     config = json.load(f)

In [26]:
OPENSCAD_EXEC = config["OPENSCAD_EXEC_PATH"]
FILE_DIR = "../common/helix_gear"
DESIGN_FILENAME = "gears_design.md"
PARAMETERS_FILENAME = "gears_parameters.json"
SCRIPT_FILENAME = "gears_script.scad"
if not os.path.exists(FILE_DIR):
    os.makedirs(FILE_DIR)

In [28]:
client = OpenAI(api_key=config['OPENAI_API_KEY'])

In [5]:
design_description = f""" 
Suppose there is a one-stage system with two helical gears with a 30 degrees pitch angle.
Gear axes are parallel and the distance between two axes is 50mm. 
The height of driven and driving gears are 20 and 15 degree respectively, the pressure angle are both 20 degrees.
The desired gear ratio of the reduction system is 1.5:1, 
Set the driving gear to the center of the plane, which is the coordinate (0, 0)
Based on the given information, computer module number, teeth number of both gears and the coordniate of the driven gear centers.
"""

In [6]:
design_messages=[
        {"role": "system", "content": design_prompt},
        {"role": "user", "content": design_description},
    ]

In [7]:
# stop_tokens = ['']
# collected_tokens = []

# res = client.chat.completions.create(
#     model=config['GPT_MODEL'],
#     messages=design_messages,
#     stream=True,
# )

# for chunk in res:
#     temp_tokens = chunk.choices[0].delta.content
#     if temp_tokens not in stop_tokens:
#         collected_tokens.append(temp_tokens)
#     else:
#         chunk_res = "".join(collected_tokens).strip()
#         print(chunk_res)
#         collected_tokens = []

res = client.chat.completions.create(
    model=config['GPT_MODEL'],
    messages=design_messages,
    temperature=0.1
)

In [8]:
print(res.choices[0].message.content)

To design a one-stage helical gear system that meets the given requirements, we'll follow these steps:

1. Determine the gear ratio.
2. Select a module number.
3. Calculate the number of teeth for both gears.
4. Determine the pitch diameters.
5. Calculate the center distance.
6. Determine the coordinates of the driven gear.

### Step 1: Determine Gear Ratio

The desired gear ratio is given as 1.5:1, which means that the driving gear must make 1.5 revolutions for every revolution of the driven gear.

Let us denote:
- $$ N_1 $$ as the number of teeth for the driving gear.
- $$ N_2 $$ as the number of teeth for the driven gear.
- $$ i $$ as the gear ratio.

Therefore, the gear ratio $$ i $$ can be described as:
$$ i = \frac{N_2}{N_1} $$

Given that $$ i = 1.5 $$, we can select an initial pair of teeth numbers that closely fit this ratio.

### Step 2: Select a Module Number

The module number is crucial to define the gear size and is related to the pitch diameter and the number of teeth vi

In [12]:
with open(os.path.join(FILE_DIR, DESIGN_FILENAME), "w") as f:
    f.write(res.choices[0].message.content)

In [15]:
parsing_description = \
f"gear 1 (driving gear): helical gear, module 2mm, teeth 20, pitch diameter 40mm, helix angle 20 degrees, height 15mm, coordinate (0, 0) " + \
f"pressure angle 20 degrees" + \
f"gear 2 (driven gear): helical gear, module 2mm, teeth 30, pitch diameter 60mm, helix angle 20 degrees, height 20mm, coordinate (50, 0) " + \
f"pressure angle 20 degrees"

In [16]:
parsing_messages = [
    {"role": "system", "content": parsing_prompt},
    {"role": "user", "content": parsing_description},
]

In [17]:
parsing_response = client.chat.completions.create(
    model=config['GPT_MODEL'],
    response_format={ "type": "json_object" },
    messages=parsing_messages,
)

In [18]:
gears_parameters = parsing_response.choices[0].message.content.strip()
print(gears_parameters)

{
  "gear 1": {
    "gear_type": "helix",
    "source": "driving",
    "coordinate": {
      "x": 0,
      "y": 0,
      "z": 0,
      "alpha": 0,
      "beta": 0,
      "gamma": 0
    },
    "unit": "mm",
    "module": 2,
    "teeth": 20,
    "height": 15,
    "pitch_d": 40,
    "helix_angle": 20,
    "pitch_angle": 0,
    "pressure_angle": 20
  },
    "gear 2": {
    "gear_type": "helix",
    "source": "driven",
    "coordinate": {
      "x": 50,
      "y": 0,
      "z": 0,
      "alpha": 0,
      "beta": 0,
      "gamma": 0
    },
    "unit": "mm",
    "module": 2,
    "teeth": 30,
    "height": 20,
    "pitch_d": 60,
    "helix_angle": 20,
    "pitch_angle": 0,
    "pressure_angle": 20
  }
}


In [19]:
with open(os.path.join(FILE_DIR, PARAMETERS_FILENAME), "w") as f:
    f.write(gears_parameters)

In [4]:
with open(os.path.join(FILE_DIR, PARAMETERS_FILENAME), "r") as f:
    parameters = json.load(f)
parameters = str(parameters)

scripting_description = f"Write a script according to provided parameters of gears \n" + parameters

In [5]:
script_messages = [
    {"role": "system", "content": scripting_prompt},
    {"role": "user", "content": scripting_description}
]

In [6]:
scripting_response = client.chat.completions.create(
    model=config['GPT_MODEL'],
    messages=script_messages,
    temperature=0
)

In [7]:
def script_postprocessing(script):
    # To remove textual perfix or suffix information in the raw script generated by GPT-4
    pattern = re.compile(r'```(?:openscad|scad)(.*?)```', re.DOTALL)
    matches = re.findall(pattern, script)
    clean_script = ""
    if len(matches) > 0:
        for m in matches:
            clean_script = clean_script + m.strip()
    else:
        clean_script = script
    return clean_script

In [8]:
script = scripting_response.choices[0].message.content.strip()
script = script_postprocessing(script)
print(script)

translate([0,0,0]) rotate([0,0,0])
gear_helix(m=2, z=20, h=15, w=20, w_helix=20);

translate([50,0,0]) rotate([0,0,0])
gear_helix(m=2, z=30, h=20, w=20, w_helix=-20);


In [9]:
with open(os.path.join(FILE_DIR, SCRIPT_FILENAME), "w") as f:
    f.write("include <gears.scad>")

with open(os.path.join(FILE_DIR, SCRIPT_FILENAME), "a") as f:
    f.write("\n" + script)

In [29]:
subprocess.Popen([OPENSCAD_EXEC, os.path.join(FILE_DIR, SCRIPT_FILENAME)])

<Popen: returncode: None args: ['D:\\openSCAD\\install\\openscad.exe', '../c...>